In [ ]:
import nltk
import numpy as np
import operator as op
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import io
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
processed_train = '/content/drive/My Drive/processed_data/data_train_t_processed.csv'
processed_dev   = '/content/drive/My Drive/processed_data/data_dev_t_processed.csv'
processed_test  = '/content/drive/My Drive/processed_data/data_test_t_processed.csv'

In [ ]:
# Loading the processed data
data_train_processed = pd.read_csv(processed_train)
data_dev_processed = pd.read_csv(processed_dev)
data_test_processed = pd.read_csv(processed_test)


In [ ]:
data_train_processed['review_id']

0           review_58145147
1           review_59303833
2           review_61916672
3         review_1000190786
4           review_59207755
                ...        
665957      review_55820729
665958    review_1001840194
665959      review_56242470
665960      review_58174196
665961    review_1006358741
Name: review_id, Length: 665962, dtype: object

In [ ]:
# Accessing the processed comments
processed_comments_train = data_train_processed['processed_commentaire']
processed_comments_dev = data_dev_processed['processed_commentaire']
processed_comments_test = data_test_processed['processed_commentaire']

In [ ]:
# Initialize a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
tfidf_vectorizer = TfidfVectorizer(
            analyzer = 'word',
            ngram_range=(1, 2)
        )

In [ ]:
# Fill NaN values with an empty string
data_train_processed['processed_commentaire'].fillna('', inplace=True)
data_dev_processed['processed_commentaire'].fillna('', inplace=True)
data_test_processed['processed_commentaire'].fillna('', inplace=True)

In [ ]:
# Fit and transform the preprocessed text data
X_train = tfidf_vectorizer.fit_transform(data_train_processed['processed_commentaire'])
X_dev = tfidf_vectorizer.transform(data_dev_processed['processed_commentaire'])
X_test = tfidf_vectorizer.transform(data_test_processed['processed_commentaire'])

In [ ]:
data_train_processed['note'].values

array([4. , 3.5, 4.5, ..., 3.5, 3. , 3.5])

In [ ]:
import pickle

# Load the TF-IDF transformed training data
with open('/content/drive/My Drive/tef_idf/tfidf_train_t.pkl', 'rb') as f:
    X_train = pickle.load(f)

# Load the TF-IDF transformed development data
with open('/content/drive/My Drive/tef_idf/tfidf_dev_t.pkl', 'rb') as f:
    X_dev = pickle.load(f)

# Load the TF-IDF transformed test data
with open('/content/drive/My Drive/tef_idf/tfidf_test_t.pkl', 'rb') as f:
    X_test = pickle.load(f)



In [ ]:
import pickle

with open('/content/drive/My Drive/tef_idf/tfidf_train_t.pkl', 'wb') as f:
    pickle.dump(X_train, f)

# For the development dataset
with open('/content/drive/My Drive/tef_idf/tfidf_dev_t.pkl', 'wb') as f:
    pickle.dump(X_dev, f)

# For the test dataset
with open('/content/drive/My Drive/tef_idf/tfidf_test_t.pkl', 'wb') as f:
    pickle.dump(X_test, f)


In [ ]:
def sparse_matrix_to_dict(sparse_mat):
    doc_dict = {}
    print("sparse_matrix_to_dict")
    cx = sparse_mat.tocoo()
    for i, j, v in zip(cx.row, cx.col, cx.data):
        if i not in doc_dict:
            doc_dict[i] = {}
        doc_dict[i][j] = v
    return doc_dict

In [ ]:
def convertir_svm_from_dict(data_dict, etiquettes):
    svm = ""
    print("convertir_svm")
    for i, features in data_dict.items():
        # Convert label to integer by multiplying by 2
        label = int(float(etiquettes[i]) * 2)
        svm += str(label) + " "

        # Trier les clés (indices de caractéristiques) dans l'ordre croissant
        sorted_features = sorted(features.items())

        for j, value in sorted_features:
            svm += f"{j + 1}:{value} "
        svm += "\n"
    return svm

In [ ]:
def convertir_svm_from_t_dict(data_dict, etiquettes):
    svm = ""
    for i, features in data_dict.items():
        # Handle missing labels by using a default label
        label = int(float(etiquettes[i]) * 2) if i < len(etiquettes) else 0
        svm += str(label) + " "
        for j, value in sorted(features.items()):
            svm += f"{j + 1}:{value} "
        svm += "\n"
    return svm


In [ ]:
def file(nom,contenu):

    # Ouvrez le fichier en mode écriture
    with io.open(nom, "w", encoding="utf-8") as f:
        # Écrivez vos lignes de string dans le fichier
        f.writelines(contenu)


In [ ]:
X_train_dict = sparse_matrix_to_dict(X_train)

X_test_dict = sparse_matrix_to_dict(X_test)


sparse_matrix_to_dict
sparse_matrix_to_dict


In [ ]:
X_dev_dict = sparse_matrix_to_dict(X_dev)


sparse_matrix_to_dict


In [ ]:
libsvm_format_data = convertir_svm_from_dict(X_train_dict, data_train_processed['note'])
file("/content/drive/My Drive/processed_data/train_t.svm", libsvm_format_data)

convertir_svm


In [ ]:
libsvm_format_data = convertir_svm_from_dict(X_dev_dict, data_dev_processed['note'])
file("/content/drive/My Drive/processed_data/dev_t.svm", libsvm_format_data)

convertir_svm


In [ ]:
# Assign a placeholder label (e.g., 0) for the test data
test_labels = [0] * len(X_test_dict)

# Convert the test data to SVM format
libsvm_format_data_test = convertir_svm_from_t_dict(X_test_dict, test_labels)

# Save the test data SVM file
file("/content/drive/My Drive/processed_data/test_t.svm", libsvm_format_data_test)


In [ ]:
#Fonction pour convertir un nombre flottant en chaîne avec une virgule pour le décimal
def float_to_comma_string(x):
    if isinstance(x, float):
        return '{:.1f}'.format(x).replace('.', ',')
    return x

#Fonction pour lire les prédictions à partir de out.txt
def read_labels_file(filepath):
    with open(filepath, 'r') as file:
        labels = [line.strip() for line in file]
    return pd.DataFrame(labels, columns=['prediction'])

#Lecture des prédictions
predictions_df = read_labels_file('/content/drive/My Drive/processed_data/res.txt')

#Convertir les prédictions en nombres, les diviser par 2, et les formater avec une virgule
predictions_df['prediction'] = pd.to_numeric(predictions_df['prediction'], errors='coerce') / 2
predictions_df = predictions_df.applymap(float_to_comma_string)


#Fusionner les prédictions avec review_id
df_test = pd.concat([data_test_processed.review_id, predictions_df], axis=1)

#Enregistrer dans un nouveau fichier texte séparé par un espace
df_test.to_csv('/content/drive/My Drive/processed_data/output_fn.txt', sep=' ', index=False, header=False)

In [ ]:
# Check that the predictions DataFrame has the correct number of rows
print(predictions_df.shape)

# Check for NaN values which might cause issues when casting to numeric
print(predictions_df.isnull().sum())

# Check the first few rows of the DataFrame
print(predictions_df.head())

# Check the first few rows of the DataFrame with review IDs
print(df_test.head())

# Check the correspondence of the number of predictions to the number of test samples
print(f"Number of test samples: {data_test_processed.shape[0]}")
print(f"Number of predictions: {predictions_df.shape[0]}")


(85832, 1)
prediction    0
dtype: int64
  prediction
0        4,0
1        4,5
2        3,5
3        5,0
4        5,0
         review_id prediction
0  review_59354742        4,0
1  review_62570109        4,5
2  review_58180650        3,5
3  review_57696986        5,0
4  review_57736972        5,0
Number of test samples: 85847
Number of predictions: 85832


In [ ]:
pip install vaderSentiment-fr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.1/187.1 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.1 MB/s eta 0:00:00
  Created wheel for vaderSentiment-fr: filename=vaderSentiment_fr-1.3.4-py3-none-any.whl size=185960 sha256=16b1703d2e1b2e029a9f91e454acc971532adeb0cdac9cc444e686e1ea72fbf2
  Stored in directory: /root/.cache/pip/wheels/a3/a0/97/307bbd5f16d2e240c69421a79a42a18797067af832ee955b09
Successfully built vaderSentiment-fr


In [ ]:
from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer

SIA = SentimentIntensityAnalyzer()

phrase = data_train_processed['processed_commentaire'].iloc[0]

score = SIA.polarity_scores(phrase)

In [ ]:
score

{'neg': 0.042, 'neu': 0.771, 'pos': 0.187, 'compound': 0.9397}

In [ ]:
def extract_sentiment_lexicon(file_path):
    sentiment_lexicon = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.split(';')
            if len(parts) >= 4:
                word = parts[1]
                sentiment = parts[3]
                if sentiment == 'positive':
                    sentiment_lexicon[word] = 1
                elif sentiment == 'negative':
                    sentiment_lexicon[word] = -1
                else:
                    sentiment_lexicon[word] = 0
    return sentiment_lexicon

# Load your sentiment lexicon
sentiment_lexicon = extract_sentiment_lexicon('/content/drive/My Drive/processed_data/Polarimots.csv')


In [ ]:
def calculate_sentiment_score(comment, sentiment_lexicon):
    words = comment.split()
    score = sum(sentiment_lexicon.get(word, 0) for word in words)
    return score

# Convert all entries to strings
data_train_processed['processed_commentaire'] = data_train_processed['processed_commentaire'].astype(str)

# Calculate sentiment scores
data_train_processed['sentiment_score'] = data_train_processed['processed_commentaire'].apply(lambda x: calculate_sentiment_score(x, sentiment_lexicon))


# Now, 'sentiment_score' can be used as a feature in your model


In [ ]:
data_train_processed['sentiment_score']

0          5
1         18
2          6
3          2
4          2
          ..
665957     2
665958     2
665959     1
665960     1
665961     4
Name: sentiment_score, Length: 665962, dtype: int64

In [ ]:
# Convert all entries to strings
data_test_processed['processed_commentaire'] = data_test_processed['processed_commentaire'].astype(str)

# Calculate sentiment scores
data_test_processed['sentiment_score'] = data_test_processed['processed_commentaire'].apply(lambda x: calculate_sentiment_score(x, sentiment_lexicon))

In [ ]:
data_test_processed['sentiment_score']

0        9
1       -1
2        1
3        1
4        1
        ..
85842    1
85843    0
85844    3
85845    5
85846    4
Name: sentiment_score, Length: 85847, dtype: int64

In [ ]:

from scipy.sparse import hstack

# Convert the TF-IDF sparse matrix to a DataFrame
tfidf_df = pd.DataFrame(X_train.toarray(), columns=tfidf_vectorizer.get_feature_names_out())


data_train_processed = data_train_processed.reset_index(drop=True)

# Concatenate the TF-IDF features with the sentiment scores
combined_features = pd.concat([tfidf_df, data_train_processed['sentiment_score']], axis=1))


In [ ]:
from sklearn.preprocessing import OneHotEncoder

tfidf = TfidfVectorizer(
    stop_words = stopwords.words("french") + stopwords.words("english"),
    analyzer = 'word',
    lowercase = True,
    ngram_range = (1,2)
)
preprocessor = ColumnTransformer(transformers=[
                ('tfidf', tfidf, "commentaire"),
                ("scaler", OneHotEncoder(), ['emoticon'])
            ])
model = make_pipeline(preprocessor, SGDClassifier(loss = "hinge", penalty = "l2"))
model.fit(data_train_processed, data_train_processed["note"])

In [ ]:
data_train_processed

,movie,review_id,name,user_id,note,commentaire,processed_commentaire,sentiment_score
0,229864,review_58145147,AM11,Z20051105131449940367618,4.0,"""Ce n'est pas ce qui se trouve en face de vous...",nest pas_ce trouver face cest tenir ct ce phra...,5
1,229864,review_59303833,this is my movies,Z20051220181957240298347,3.5,"(...) En soi, je n'ai rien contre ce genre de ...",soi nai rien contre genre rcit hyper codifi qu...,18
2,229864,review_61916672,Shephard69,Z20121226180301760912584,4.5,"Pour sa troisième réalisation après ""Tron, l'h...",troisim ralisation apr tron lhritage oblivion ...,6
3,229864,review_1000190786,Tayop,Z20121022182542230468657,3.0,Joseph Kosinski offre de nouvelles images ébou...,joseph kosinski offrir nouveau image bouriffan...,2
4,229864,review_59207755,ned123,Z20061211223247607682175,3.5,J'ai vu un film... qui traite d'un drame humai...,jai voir film traite dun drame humain plus hau...,2
...,...,...,...,...,...,...,...,...
665957,245551,review_55820729,Wally L,Z20121109004006553333277,4.0,super realisation et quelques bon acteurs. ce...,super realisation quelque bon acteur ce ville ...,2
665958,245551,review_1001840194,r@ch,Z20100726110927837577369,3.0,Film visuellement très beau et mis en musique ...,film visuellemer trs beau mettre musiqu rap am...,2
665959,245551,review_56242470,Kaiser D.,Z20170721223303710182536,3.5,parti d'une histoire qui parait pour nous bana...,partir dune histoire parer banal tr bon film t...,1
665960,245551,review_58174196,Florent A.,Z20161204004443590736237,3.0,"Je suis mitigé sur ce film.d un coté,j ai plut...",mitig filmd cotj plutot accrochmer peu mal avo...,1
